In [17]:
!pip install google-generativeai python-pptx

In [18]:
import google.generativeai as genai
from pptx import Presentation

# Set up Gemini API (free tier)
genai.configure(api_key="API-KEY")


In [19]:
def generate_slides_content(summary):
    """
    Generates structured PowerPoint slide content using Gemini API.
    Returns formatted text with slide titles and bullet points.
    """
    prompt = f"""Create an 8-slide PowerPoint presentation from this research paper summary.
    Ensure every slide follows this structure:

    Slide 1: Title Slide
    - **Title**: [Research Paper Title]
    - **Presented by:**: "Presented by: Deep Drillers"

    Slide 2: Introduction
    - Background of the study
    - Research objectives
    - Key questions addressed

    Slide 3: Methodology
    - Research design
    - Data collection methods
    - Analysis techniques

    Slide 4: Key Findings
    - Main result 1
    - Main result 2
    - Main result 3

    Slide 5: Analysis
    - Interpretation of results
    - Comparison with previous work
    - Statistical significance

    Slide 6: Implications
    - Theoretical implications
    - Practical applications
    - Policy recommendations

    Slide 7: Limitations
    - Study limitations 1
    - Study limitations 2
    - Future research directions

    Slide 8: Conclusion
    - Summary of key points
    - Final remarks
    - Acknowledgments

    Paper Summary: {summary}

    **Ensure slides are formatted with 'Slide X: Title' followed by bullet points ('- ').**
    """

    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(prompt)

    slide_content = response.text.strip()  # Ensure clean formatting

    # Remove the asterisks from the slide content
    slide_content = slide_content.replace("**", "")

    print("Generated Slide Content:\n", slide_content)
    return slide_content


In [20]:
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor
import os

THEME_DIR = "themes"

def apply_theme(theme_name):
    """Load presentation with selected theme template"""
    template_path = os.path.join(THEME_DIR, f"{theme_name}.pptx")

    if not os.path.exists(template_path):
        raise ValueError(f"Theme {theme_name} not found")

    return Presentation(template_path)

def parse_ai_content(ai_output):
    """Convert raw AI text output to structured format"""
    slides = []
    current_slide = {"title": "", "bullets": []}

    for line in ai_output.split("\n"):
        line = line.strip()
        if line.startswith("Slide"):
            if current_slide["title"]:
                slides.append(current_slide)
                current_slide = {"title": "", "bullets": []}
            current_slide["title"] = line.split(": ")[1]
        elif line.startswith("-"):
            current_slide["bullets"].append(line[2:])

    if current_slide["title"]:
        slides.append(current_slide)

    return slides

def style_slide(slide, theme):
    """Applies theme-specific styling"""
    # Title styling
    title = slide.shapes.title
    title.text_frame.paragraphs[0].font.size = theme["title_size"]
    title.text_frame.paragraphs[0].font.color.rgb = theme["title_color"]

    # Content styling
    for shape in slide.shapes:
        if shape.has_text_frame:
            for paragraph in shape.text_frame.paragraphs:
                paragraph.font.name = theme["body_font"]
                paragraph.font.size = theme["body_size"]
                paragraph.line_spacing = 1.2

THEMES = {
    "corporate": {
        "title_size": Pt(60),
        "title_color": RGBColor(13, 27, 42),
        "body_font": "Calibri",
        "body_size": Pt(20)
    },
    "academic": {
        "title_size": Pt(32),
        "title_color": RGBColor(0, 0, 0),
        "body_font": "Times New Roman",
        "body_size": Pt(18)
    }
}

def create_presentation_4(content, theme_name, filename="research_presentation.pptx"):
    """Main function to create styled presentation"""
    # Parse raw AI content
    structured_content = parse_ai_content(content)

    # Load theme template
    prs = apply_theme(theme_name)
    theme = THEMES.get(theme_name, THEMES[theme_name])

    # Create slides
    for slide_data in structured_content:
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        title = slide.shapes.title
        title.text = slide_data["title"]

        content_box = slide.shapes.placeholders[1]
        for bullet in slide_data["bullets"]:
            p = content_box.text_frame.add_paragraph()
            p.text = bullet
            p.level = 0

        style_slide(slide, theme)

    # Add footer
    for slide in prs.slides:
        txBox = slide.shapes.add_textbox(Inches(0.5), Inches(6.5), Inches(9), Inches(0.5))
        tf = txBox.text_frame
        tf.text = "Generated by AI Research Assistant | Confidential"
        tf.paragraphs[0].font.size = Pt(12)
        tf.paragraphs[0].font.color.rgb = RGBColor(150, 150, 150)

    prs.save(filename)
    return filename



In [22]:
# Example Usage
if __name__ == "__main__":
    paper_summary = """ *Quantum Communication-Based Edge Intelligence Framework for Wearable Health IoT Devices*    ## *1. Introduction*    ### *1.1. The Rise of Healthcare IoT (H-IoT)*   The rapid growth of *Healthcare Internet of Things (H-IoT)* has revolutionized the medical industry by enabling *real-time patient monitoring, remote diagnostics, and intelligent decision-making. Devices such as **wearable heart monitors, smart glucose sensors, and AI-driven health assistants* generate vast amounts of sensitive patient data, which needs to be securely transmitted to hospitals and medical institutions.    However, the deployment of *H-IoT networks* comes with significant *security challenges. Traditional cryptographic techniques, such as **RSA (Rivest-Shamir-Adleman) and AES (Advanced Encryption Standard), are becoming increasingly vulnerable due to the rapid advancement of **quantum computing. **Shor’s Algorithm, a quantum algorithm, can potentially break classical encryption systems, posing **a major cybersecurity threat* to healthcare networks.    ### *1.2. Security Threats in H-IoT Networks*   Healthcare IoT devices communicate over *wireless networks, making them susceptible to cyber threats such as:   - **Eavesdropping and Man-in-the-Middle (MitM) Attacks* – Hackers can intercept medical data.   - *Data Tampering and Spoofing* – Unauthorized modification of patient records.   - *Denial-of-Service (DoS) Attacks* – Overloading healthcare servers, causing downtime.   - *Quantum-based Cryptanalysis* – Future quantum computers could break encryption keys.    ### *1.3. The Need for Quantum Cryptography and Edge Intelligence*   To address these security challenges, this research proposes a *Quantum Communication-Based Edge Intelligence Framework* for H-IoT networks. This framework integrates:   ✔ *Quantum Key Distribution (QKD)* – A theoretically unbreakable encryption method using *BB84 and E91 protocols.   ✔ **Machine Learning (ML)-Based Intrusion Detection System (IDS)* – Detects cyber threats in H-IoT networks.   ✔ *Edge Computing* – Reduces latency and improves real-time healthcare data processing.    ---  ## *2. Background and Related Work*    ### *2.1. Quantum Cryptography in Healthcare*   *Quantum Cryptography* is an advanced technique that utilizes the *principles of quantum mechanics* to encrypt and securely transmit data. The two primary *QKD protocols* used in this research are:   - *BB84 Protocol* – Uses quantum bits (qubits) and polarization states to create a secure key exchange.   - *E91 Protocol* – Uses quantum entanglement to create an unhackable key distribution system.    ### *2.2. Machine Learning for Intrusion Detection*   Intrusion Detection Systems (IDS) rely on *ML algorithms* to classify network traffic as either *normal or malicious. The study evaluates the performance of:   - **Random Forest (RF)*   - *K-Nearest Neighbors (KNN)*   - *Naïve Bayes (NB)*   - *Gradient Boosting Machine (GBM)*    Among these, *Random Forest achieved the highest accuracy (90.51%), making it the preferred choice for securing the H-IoT network.    ### **2.3. Edge Computing for Healthcare IoT*   Traditional *cloud-based healthcare systems* introduce latency due to *data transfer* and *processing bottlenecks. By implementing **edge computing, patient data is **processed locally* on IoT devices, reducing network congestion and improving efficiency.    ---  ## *3. Proposed Framework for Secure H-IoT Communication*    ### *3.1. System Overview*   The proposed *Quantum Communication-Based Edge Intelligence Framework* consists of *four key layers:    1️⃣ **Wearable Health Edge Devices Layer*   ✔ Collects *real-time patient health data* using wearable devices.   ✔ Preprocesses data *locally* using edge computing.    2️⃣ *Network Intrusion Detection Layer*   ✔ Uses *ML models* (Random Forest, KNN, NB, GBM) to detect cyberattacks.   ✔ *RF model achieved 90.51% accuracy* in identifying malicious traffic.    3️⃣ *Quantum Communication Layer*   ✔ Encrypts medical data using *BB84 and E91 protocols.   ✔ Ensures **secure end-to-end transmission* from devices to hospitals.    4️⃣ *Hospital Layer*   ✔ Monitors *real-time patient data* securely.   ✔ Reduces *latency* using *edge-based processing* instead of cloud reliance.    ### *3.2. Encryption and Secure Data Transmission*   - *BB84 Quantum Key Distribution (QKD)* generates *random encryption keys, preventing eavesdropping.   - **Quantum Entanglement (E91 Protocol)* ensures *tamper-proof communication.   - **Edge Computing* processes data *locally* before encryption, reducing computational overhead.    ---  ## *4. Experimental Setup and Results*    ### *4.1. Dataset and Implementation*   - The *CICIDS-2017 dataset* was used to train the ML-based IDS.   - Experiments were conducted using *Python, TensorFlow, and Quantum Key Simulators.    ### **4.2. Intrusion Detection System (IDS) Performance*   The study tested four ML models for IDS:    | Model          | Accuracy (%) | |---------------|-------------| | *Random Forest (RF)*  | *90.51* | | K-Nearest Neighbors (KNN) | 86.34 | | Naïve Bayes (NB) | 78.12 | | Gradient Boosting (GBM) | 88.91 |  *Conclusion:* *RF outperformed all other models, making it the best choice for H-IoT security.    ### **4.3. Quantum Cryptography Evaluation*   | *Protocol*  | *Data Transfer Speed* | *Security Level* | |--------------|------------------|----------------| | *BB84* | *Higher throughput* | Moderate Security | | *E91* | *Stable real-time encryption* | High Security |  *Conclusion:* BB84 is suitable for *fast data transfers, while E91 is better for **high-security applications.    ---  ## **5. Discussion and Future Improvements*    ### *5.1. Advantages of the Proposed Framework*   ✔ *Highly Secure Communication* – Uses *quantum encryption* to protect patient data.   ✔ *Fast Intrusion Detection* – ML-based *IDS detects threats in real time.   ✔ **Low Latency Processing* – *Edge computing* minimizes cloud dependency.    ### *5.2. Limitations and Challenges*   - *High Computational Cost:* Quantum encryption methods are *resource-intensive.   - **Scalability Issues:* Deploying *large-scale QKD networks* remains a challenge.   - *Integration with Legacy Systems:* Older medical systems need *compatibility adjustments.    ### **5.3. Future Work*   - *Optimize QKD protocols* for *faster key generation.   - **Implement Blockchain Technology* to enhance *security and decentralization.   - **Deploy AI-powered anomaly detection* to improve *threat identification.    ---  ## **6. Conclusion*    This research presents a *Quantum Communication-Based Edge Intelligence Framework* for *securing Healthcare IoT networks. By integrating **Quantum Cryptography (BB84 & E91 protocols), ML-based Intrusion Detection (RF, KNN, NB, GBM), and Edge Computing, the system significantly enhances **data security, real-time processing, and cyberattack prevention.    ### **Key Takeaways*   ✔ *Quantum Key Distribution (QKD)* ensures *secure encryption* of patient data.   ✔ *Random Forest (RF) achieved 90.51% accuracy* in detecting network intrusions.   ✔ *BB84 protocol is ideal for fast transmission, while **E91 is suited for high-security applications.   ✔ **Edge Computing reduces latency, enabling **real-time healthcare monitoring.    ### **Final Thought*   The proposed framework provides a *secure, AI-driven, and quantum-enhanced* solution for *future healthcare IoT networks, paving the way for **safer and smarter medical data transmission. """
    chosen_theme = "academic"
    # Generate slide content
    print("Generating slide content...")
    content = generate_slides_content(paper_summary)
    print("Content generated!\n")

    # Create PowerPoint
    print("Creating presentation...")
    output_file = create_presentation_4(content,chosen_theme)
    print(f"Presentation saved as {output_file}")

    # If using Google Colab, download the file
    try:
        from google.colab import files
        files.download(output_file)
    except ImportError:
        print("Run this in Colab to download the file automatically.")


Generating slide content...
Generated Slide Content:
 Slide 1: Title Slide

- Title: Quantum Communication-Based Edge Intelligence Framework for Wearable Health IoT Devices
- Presented by: Deep Drillers

Slide 2: Introduction

- Background: Rise of Healthcare IoT (H-IoT) and security challenges due to quantum computing.
- Research objectives: Develop a secure framework using quantum cryptography and edge intelligence for H-IoT networks.
- Key questions addressed:
    - How to protect healthcare data from quantum-based threats?
    - How to detect and prevent cyberattacks in H-IoT networks?
    - How to reduce latency and improve real-time data processing?

Slide 3: Methodology

- Research design: Experimental study using ML models and quantum key simulation.
- Data collection methods: CICIDS-2017 dataset for intrusion detection.
- Analysis techniques: Performance evaluation of ML models, security analysis of quantum cryptography protocols.

Slide 4: Key Findings

- Main result 1: Rando

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>